# IMDB 영화 리뷰 감성 분석 - Conv1D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

### Conv1D으로 IMDB 리뷰 감성 분석
- 단어 빈도수 : 10,000 (88,584)
- 문장의 단어수 : 500 (2,494)
- test data중 40%(10,000개)는 검증용으로

In [2]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [6]:
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

- Case 1) Conv1D x 2, Maxpooling1D x 2, Dropout, GlobalMaxPooling1D
    - embedding dim : 100

In [7]:
model1 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [8]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [9]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.5406 - accuracy: 0.6987
Epoch 00001: val_loss improved from inf to 0.35337, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 18s 20ms/step - loss: 0.5406 - accuracy: 0.6987 - val_loss: 0.3534 - val_accuracy: 0.8435
Epoch 2/30
195/196 [============================>.] - ETA: 0s - loss: 0.2811 - accuracy: 0.8863
Epoch 00002: val_loss improved from 0.35337 to 0.32015, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 4s 19ms/step - loss: 0.2811 - accuracy: 0.8862 - val_loss: 0.3202 - val_accuracy: 0.8626
Epoch 3/30
195/196 [============================>.] - ETA: 0s - loss: 0.2068 - accuracy: 0.9205
Epoch 00003: val_loss did not improve from 0.32015
196/196 [==============================] - 3s 17ms/step - loss: 0.2068 - accuracy: 0.9205 - val_loss: 0.3346 - val_accuracy: 0.8615
Epoch 4/30
194/196 [============================>.] - ETA: 0s - loss: 0.1549 - accur

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 18s 38ms/step - loss: 0.0000e+00 - accuracy: 0.5000


[0.0, 0.5]

- Case 2) Conv1D + LSTM

In [10]:
from tensorflow.keras.layers import LSTM

In [12]:
model2 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 494, 64)           44864     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 123, 64)          0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100)               66000     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                      

In [13]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [14]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.4399 - accuracy: 0.7742
Epoch 00001: val_loss improved from inf to 0.31025, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 11s 30ms/step - loss: 0.4399 - accuracy: 0.7742 - val_loss: 0.3102 - val_accuracy: 0.8662
Epoch 2/30
195/196 [============================>.] - ETA: 0s - loss: 0.2275 - accuracy: 0.9125
Epoch 00002: val_loss did not improve from 0.31025
196/196 [==============================] - 5s 26ms/step - loss: 0.2275 - accuracy: 0.9126 - val_loss: 0.3148 - val_accuracy: 0.8634
Epoch 3/30
194/196 [============================>.] - ETA: 0s - loss: 0.1700 - accuracy: 0.9378
Epoch 00003: val_loss did not improve from 0.31025
196/196 [==============================] - 5s 25ms/step - loss: 0.1703 - accuracy: 0.9377 - val_loss: 0.3462 - val_accuracy: 0.8704
Epoch 4/30
194/196 [============================>.] - ETA: 0s - loss: 0.1313 - accuracy: 0.9530
Epoch 00004: val_loss di

In [16]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 4s 8ms/step - loss: 0.3108 - accuracy: 0.8676


[0.31077826023101807, 0.8676000237464905]

- Case 3) Conv1D + Dense

In [19]:
model3 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100,activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_5 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_6 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 100)              

In [20]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-fcn.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [21]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
196/196 [==============================] - ETA: 0s - loss: 0.5784 - accuracy: 0.6700
Epoch 00001: val_loss improved from inf to 0.36527, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 6s 22ms/step - loss: 0.5784 - accuracy: 0.6700 - val_loss: 0.3653 - val_accuracy: 0.8360
Epoch 2/30
194/196 [============================>.] - ETA: 0s - loss: 0.2971 - accuracy: 0.8767
Epoch 00002: val_loss improved from 0.36527 to 0.27278, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 4s 19ms/step - loss: 0.2969 - accuracy: 0.8768 - val_loss: 0.2728 - val_accuracy: 0.8872
Epoch 3/30
195/196 [============================>.] - ETA: 0s - loss: 0.2083 - accuracy: 0.9190
Epoch 00003: val_loss improved from 0.27278 to 0.26375, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 4s 20ms/step - loss: 0.2086 - accuracy: 0.9189 - val_loss: 0.2637 - val_accuracy: 0.8926
Epoch 4/30
193/196 [============

In [22]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 4ms/step - loss: 0.2682 - accuracy: 0.8901


[0.26820850372314453, 0.8900666832923889]